Set up

In [2]:
import psycopg2
from sqlalchemy import create_engine
import os
import pandas as pd

In [8]:
# Set the working directory
current_dir = os.getcwd()
print('current_dir: ',current_dir)
parent_dir = os.path.dirname(current_dir)

os.chdir(parent_dir)

print('Current working directory:', os.getcwd())

current_dir:  c:\Users\Utente\Documents\GitHub\SE4GEO-Lab\db_helper
Current working directory: c:\Users\Utente\Documents\GitHub\SE4GEO-Lab


In [ ]:
from se4g_helper import download_request, build_dataframe, update_dataset, update_dashboard_dataset
# Download and get the dataframe file name
dir = download_request(folder_out = 'data_prova')

# Build the dataframe with the required structure
df = build_dataframe(dir, folder_out = 'data_prova')

# Update the final dataset
new_df = update_dataset(df, folder_out = 'data_prova')
# Update the dashboard dataset 
update_dashboard_dataset(df,folder_out = 'data_prova')

In [20]:
ip = '192.168.30.19'
ip = 'localhost'
# Connect to the database
conn = psycopg2.connect(
    host = ip,
    database = "se4g",
    user = "postgres",
    password = "carIs3198"
)

In [15]:
def insert_data(table_name, conn, df):
    cur = conn.cursor()
    
    # Specify the table name and column names
    df_columns = ['station_code', 
                'station_name', 
                'station_altitude', 
                'network_countrycode', 
                'pollutant', 
                'value_datetime_begin',
                'value_datetime_end',
                'value_datetime_updated',
                'value_numeric']

    # Iterate over the DataFrame rows and insert data row by row
    for _, row in df.iterrows():
        # Generate the SQL INSERT statement
        insert_statement = f"INSERT INTO {table_name} ({', '.join(df_columns)}) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = tuple(row[col] for col in df_columns)

        # Execute the INSERT statement
        cur.execute(insert_statement, values)

    
    # Commit the changes and close the cursor 
    conn.commit()
    cur.close()

# Call the insert_data function with the table name and connection
insert_data('se4g_pollution', conn, new_df)


In [ ]:
# Dislpay table
query = "SELECT * FROM se4g_pollution"
df = pd.read_sql_query(query, conn)

print(df)

In [29]:
# Create a cursor
cur = conn.cursor()

# Specify the datetime value
datetime_value_begin = '2023-05-29 08:00:00+01:00'
datetime_value_end = '2023-05-29 12:00:00+01:00'

# Specify the network countrycode
network_countrycode = 'AD'

# Construct the SQL query with the conditions
query = f"SELECT pollutant, value_datetime_begin, value_datetime_end FROM se4g_pollution WHERE value_datetime_end > %s AND value_datetime_end < %s AND network_countrycode = %s"

# Execute the query
cur.execute(query, (datetime_value_begin, datetime_value_end, network_countrycode))

# Fetch all rows from the result set
rows = cur.fetchall()

# Get the column names
columns = [desc[0] for desc in cur.description]

# Create a pandas DataFrame from the rows and columns
df = pd.DataFrame(rows, columns=columns)

# Close the cursor
cur.close()

print(df)

   pollutant    value_datetime_begin         value_datetime_end
0        SO2  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
1        SO2  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
2        SO2  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
3         NO  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
4         NO  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
5         NO  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
6        NO2  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
7        NO2  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
8        NO2  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
9         CO  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
10        CO  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
11        CO  2023-05-29 09:00:00+00  2023-05-29 11:00:00+01:00
12      PM10  2023-05-29 07:00:00+00  2023-05-29 09:00:00+01:00
13      PM10  2023-05-29 08:00:00+00  2023-05-29 10:00:00+01:00
14      PM10  2023-05-29 09:00:00+00  20

In [ ]:
conn.close()

In [ ]:
cur.execute('''UPDATE pollutant_detection
 (station_code TEXT , 
 name_pollutant TEXT, 
 data_time_begin DATE, 
 data_time_end DATE,
 data_time_update DATE,
 value_pollutant DECIMAL,
 PRIMARY KEY (name_pollutant, station_code, data_time_begin,data_time_end ))''')